# Tidy3D mode solver

In [ ]:
import gdsfactory.simulation.gtidy3d as gt

In [ ]:
strip = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, t_wg=0.22, t_slab=0.0)
strip

In [ ]:
strip.plot_Ex(0) # TE
strip.plot_Ey(1) # TM

In [ ]:
rib = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, t_wg=0.22, t_slab=0.15)
rib.plot_index()

In [ ]:
rib.plot_Ex(index=0)
rib.plot_Ey(index=0)

## Bend modes

In [ ]:
strip_bend = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, t_wg=0.22, t_slab=0.0, bend_radius=3)

In [ ]:
# lets plot the fundamental TE mode
strip_bend.plot_Ex(0)
strip_bend.plot_Ey(0)

In [ ]:
# lets plot the fundamental TM mode
strip_bend.plot_Ex(1)
strip_bend.plot_Ey(1)

## Bend loss

To compute the bend loss you can calculate the mode overlap of the original mode and the bent mode.

FIXME! this does not work yet

In [ ]:
import numpy as np

index = 0 # TE mode

mode1_Ey = strip.Ey[index]
mode1_Ez = strip.Ez[index]
mode1_Hz = strip.Hz[index]
mode1_Hy = strip.Hy[index]

mode2_Ey = strip_bend.Ey [index]
mode2_Ez = strip_bend.Ez [index]
mode2_Hz = strip_bend.Hz [index]
mode2_Hy = strip_bend.Hy [index]
                
integrand = (
    np.conj(mode1_Ey) * mode2_Hz
    - np.conj(mode1_Ez) * mode2_Hy
    + mode2_Ey * np.conj(mode1_Hz)
    - mode2_Ez * np.conj(mode1_Hy)
)

# Compute integral
#t_sim = strip.t_sim
#w_sim = strip.w_sim
#wint = np.linspace(-w_sim/2, +w_sim/2, int(w_sim*strip.resolution))
#tint = np.linspace(-t_sim/2, +t_sim/2, int(t_sim*strip.resolution))
#integral = np.trapz(np.trapz(integrand, wint, axis=0), tint, axis=0)
integral = np.trapz(np.trapz(integrand, axis=0), axis=0)
abs(integral)